# NCA5 Hydrogen Senky Diagram
Looking at data from Princeton Net-Zero America: https://netzeroamerica.princeton.edu/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.sankey import Sankey

plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'

GWh_per_PJ = 277.778
TJ_per_PJ = 1000

In [2]:
df20 = pd.read_csv('data/nzap-data-2050.csv')
print(len(df20.index))

6438


In [3]:
df20 = df20[ df20['scenario'] == 'E+RE+']
print(len(df20.index))

1075


In [4]:
df20

,filter_level_1,filter_level_2,filter_level_3,variable_name,unit,value,scenario,year,geo
5,MACRO RESULTS,Primary energy,NaN,Biomass,EJ,12.27907,E+RE+,2050,national
6,MACRO RESULTS,Primary energy,NaN,Coal and Coke,EJ,0.00074,E+RE+,2050,national
15,MACRO RESULTS,Primary energy,NaN,Geothermal,EJ,0.10748,E+RE+,2050,national
21,MACRO RESULTS,Primary energy,NaN,Hydro,EJ,1.06770,E+RE+,2050,national
27,MACRO RESULTS,Primary energy,NaN,Natural Gas,EJ,0.02198,E+RE+,2050,national
...,...,...,...,...,...,...,...,...,...
6408,IMPACTS,Health,Cumulative avoided premature deaths from air p...,Cumulative avoided premature deaths from air p...,avoided deaths,15307.23738,E+RE+,2050,national
6415,IMPACTS,Health,Cumulative avoided premature deaths from air p...,Cumulative avoided premature deaths from air p...,avoided deaths,2721.08958,E+RE+,2050,national
6420,IMPACTS,Health,Cumulative avoided premature deaths from air p...,Cumulative avoided premature deaths from air p...,avoided deaths,2242.57844,E+RE+,2050,national
6428,IMPACTS,Health,Cumulative avoided premature deaths from air p...,Cumulative avoided premature deaths from air p...,avoided deaths,1513.73756,E+RE+,2050,national


In [5]:
# Take Princeton's input csv file and prep table with all needed values for Senky diagram.
def prep_dt(year=2050, scenario='E+RE+'):
    
    ### Read and get scenario
    print(f"\n\n{year}: {scenario}")
    df = pd.read_csv(f'data/nzap-data-{year}.csv')

    df = df[ df['scenario'] == scenario]
    
    
    
    ### Make new df for skimmed values
    cols = df.columns
    dic = {col : [] for col in cols}
    df1 = pd.DataFrame(dic)
    
    
    
    ### Skim all H2 related rows
    for idx in df.index:
        keep = False
        for col in cols:
            
            val = df.loc[idx, col]
            if type(val) == float or type(val) == np.float64 or type(val) == np.int64:
                continue
            
            if 'hydrogen' in val or 'Hydrogen' in val or 'H2' in val:
                keep = True
                break
        if keep:
            df1 = df1.append(df.loc[idx])
    
    df1.to_csv('tmp1.csv')
    
    
    
    ### Reduce to H2 energy related rows
    df1 = df1[ df1['unit'].isin(['PJ', 'TJ', 'GWh']) ]
    df1.to_csv('tmp2.csv')
    #print(df1)
    
    
    
    ### All values to GWh
    new_vals = []
    for idx in df1.index:
        val = df1.loc[idx, 'value']
        unit = df1.loc[idx, 'unit']
        if unit == 'GWh':
            new_vals.append(val)
        elif unit == 'PJ':
            new_vals.append(val * GWh_per_PJ)
        elif unit == 'TJ':
            new_vals.append(val / TJ_per_PJ * GWh_per_PJ)
        else:
            print(f"Unit was not considered in list, please fix this. Unit == {unit}")
            exit()
    df1['annual flow (GWh)'] = new_vals
    df1['annual flow (EJ)'] = df1['annual flow (GWh)'] / GWh_per_PJ / 1000.
    df1.to_csv('tmp3a.csv')
    #print(df1)

    
    
    ### Stacked bar comparison against PNZA's figures on slide 194 w/ production and use in EJ
    #stacked_plot(year, scenario, df1, 'Production')
    #stacked_plot(year, scenario, df1, 'Uses')
    
    

    ### Get conversion efficiencies and input energy
    effs = get_conversion_df()
    #print(effs)
    
    
    
    ### Map production conversion table to main data frame
    # NOTE: While SMR is in the datatable, SMR is skipped in the H2 production slides for 2050 (194)
    D = {
        'Production - BECCS-H2' : 'BECCS hydrogen production -> hydrogen blend',
        'Production - ATR' : 'autothermal reforming hydrogen production w/ccu -> hydrogen blend',
        'Production - Electrolysis' : 'central-station hydrogen electrolysis'
    }
    # Silly difference in name nomenclature
    S = {
        'E+RE+' : 'E+ RE+',
        'E+RE-' : 'E+ RE-',
        'REF' : 'REF',
    }
    assert(scenario in S.keys())
    
    
    ### Create main table
    # Input energy types
    inputs = []
    for tech, val in D.items():
        #print(tech)
        for idx in effs.index:
            T = effs.loc[idx, 'tech']
            #print(T)
            if T != val:
                continue
            if effs.loc[idx, 'unit'] != 'energy in/energy out':
                continue
            tech_in = effs.loc[idx, 'blend_in']
            if tech_in not in inputs:
                inputs.append(tech_in)
    print(f"Input techs: {inputs}")
    
    # Dict for new DF
    input_d = {
        'tech' : ['' for _ in range(len(D.keys()))],
        'H2 production (GWh)' : [0 for _ in range(len(D.keys()))],
    }
    for IN in inputs:
        input_d[f"{IN} (in/out)"] = [0 for _ in range(len(D.keys()))]
    
    # Populate dic
    i = 0
    for tech, val in D.items():
        input_d['tech'][i] = tech
        for idx in effs.index:
            T = effs.loc[idx, 'tech']
            #print(T)
            if T != val:
                continue
            if effs.loc[idx, 'unit'] != 'energy in/energy out':
                continue
            tech_in = effs.loc[idx, 'blend_in']
            conversion = effs.loc[idx, S[scenario]]
            input_d[f"{tech_in} (in/out)"][i] = conversion
        
        # Get row with current tech (ensure only 1 row is selected)
        row = df1['variable_name'] == tech
        assert(row.sum() == 1)
        prod = df1.loc[ row, 'annual flow (GWh)'].sum()
        input_d['H2 production (GWh)'][i] = prod
        
        i += 1
    dfx = pd.DataFrame(input_d)
    
    # Calculate total input energy
    dfx['input (GWh)'] = np.zeros(len(dfx.index))
    dfx['other output (GWh)'] = np.zeros(len(dfx.index))
    for IN in inputs:
        dfx[f"{IN} (GWh input)"] = np.where(dfx[f"{IN} (in/out)"] >= 0, dfx[f"{IN} (in/out)"], 0)  * dfx['H2 production (GWh)']
        dfx['other output (GWh)'] += np.where(dfx[f"{IN} (in/out)"] < 0, -1 * dfx[f"{IN} (in/out)"], 0)  * dfx['H2 production (GWh)']
        dfx['input (GWh)'] += dfx[f"{IN} (GWh input)"]
    dfx['losses (GWh)'] = dfx['input (GWh)'] - dfx['H2 production (GWh)'] - dfx['other output (GWh)']
    dfx['eff (%)'] = (dfx['H2 production (GWh)'] + dfx['other output (GWh)']) / dfx['input (GWh)'] * 100
    
    # Print summary outputs
    
    
    
    
    return dfx


def stacked_plot(year, scenario, df, col):
    fig, ax = plt.subplots()
    plt.title(f"{year}, {scenario}: {col}")
    
    df1a = df[ df['filter_level_3'] == col ]
    df1a = df1a.set_index('variable_name')
    dT = df1a[['annual flow (EJ)',]].T
    dT.plot(kind='bar', stacked=True, ax=ax)

    
    #d.plot(kind='bar', ax=f.gca())
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    ax.set_ylabel('H2 Quantity (EJ)')
    ax.xaxis.set_ticklabels([])
    plt.tight_layout()
    plt.savefig(f"plots/{year}_{scenario}_{col}.png")
    plt.show()
    return


def get_conversion_df():
    effs = pd.read_excel('data/NZA_Annex_A3_-_Inputs_catalog_for_EER_modeling.xlsx', sheet_name='conversion_efficiency')
    
    # fill in NaNs from previously merged cells
    p_dic = {}
    for col in ['tech', 'zone', 'vintage', 'unit']:
        p_dic[col] = ''
    for idx in effs.index:
        for col in p_dic.keys():
            val = effs.loc[idx, col]
            if type(val) != str and np.isnan(val):
                effs.loc[idx, col] = p_dic[col]
                #print('replaced', val, type(val))
            else:
                p_dic[col] = val
    
    return effs
    
    
    

#dfx = prep_dt(2050, 'E+RE+')
dfx = prep_dt(2050, 'E+RE-')



2050: E+RE-
Input techs: ['biomass blend - solids', 'electricity', 'pipeline gas blend']


In [6]:
dfx

,tech,H2 production (GWh),biomass blend - solids (in/out),electricity (in/out),pipeline gas blend (in/out),input (GWh),other output (GWh),biomass blend - solids (GWh input),electricity (GWh input),pipeline gas blend (GWh input),losses (GWh),eff (%)
0,Production - BECCS-H2,1.260784e+06,1.775,-0.027,0.000,2.237892e+06,34041.173535,2.237892e+06,0.000000,0.0000,943066.585340,57.859155
1,Production - ATR,7.862391e+05,0.000,0.048,1.203,9.835851e+05,0.000000,0.000000e+00,37739.475523,945845.6053,197346.007423,79.936051
2,Production - Electrolysis,2.721921e+05,0.000,1.282,0.000,3.489502e+05,0.000000,0.000000e+00,348950.209510,0.0000,76758.158410,78.003120
